In [2]:
import os, codecs, nltk
from nltk.corpus import wordnet

In [3]:
corpus_root = "books"

In [4]:
book_file = "king-james-bible-processed.txt"

In [22]:
book = None

with codecs.open(os.path.join(corpus_root,book_file),encoding="utf8") as f:
    book = f.read()

In [23]:
book[:100]

'In the beginning God created the heaven and the earth.\r\nAnd the earth was without form, and void; an'

In [24]:
book_sents = nltk.sent_tokenize(book)

In [29]:
book_tokens = [t for t in nltk.word_tokenize(book)]

In [31]:
book_tokens[:15]

['In',
 'the',
 'beginning',
 'God',
 'created',
 'the',
 'heaven',
 'and',
 'the',
 'earth',
 '.',
 'And',
 'the',
 'earth',
 'was']

In [33]:
book_pos = nltk.pos_tag(book_tokens)

In [38]:
book_pos[:15]

[('In', 'IN'),
 ('the', 'DT'),
 ('beginning', 'NN'),
 ('God', 'NNP'),
 ('created', 'VBD'),
 ('the', 'DT'),
 ('heaven', 'NN'),
 ('and', 'CC'),
 ('the', 'DT'),
 ('earth', 'NN'),
 ('.', '.'),
 ('And', 'CC'),
 ('the', 'DT'),
 ('earth', 'NN'),
 ('was', 'VBD')]

In [53]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [43]:
wnl = nltk.WordNetLemmatizer()

In [54]:
book_lems = [wnl.lemmatize(t,pos=get_wordnet_pos(p)) for t,p in book_pos]

In [55]:
book_lems[:15]

['In',
 'the',
 'beginning',
 'God',
 'create',
 'the',
 'heaven',
 'and',
 'the',
 'earth',
 '.',
 'And',
 'the',
 'earth',
 'be']